In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch.optim as optim
import random
import time
import json
import copy

from load_data import load_dataset_bert

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #embedded = [batch size, sent len, emb dim]
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #hidden = [batch size, hid dim]
        output = self.out(hidden)
        #output = [batch size, out dim]
        return output

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label.long())
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label.long())
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def multi_models(model, train_iterator, valid_iterator,  test_iterator, num = 5, N_EPOCHS = 50, early_stopping = 5):
    init_model = copy.deepcopy(model)
    res = {}
    res["test_loss"] = []
    res["test_acc"] = []
    ### begin training process
    for i in range(num):
        print('Begin training model %s'%i)
        res["model_%s"%i] = {}
        res["model_%s"%i]["time"] = []
        res["model_%s"%i]["train_loss"] = []
        res["model_%s"%i]["val_loss"] = []
        res["model_%s"%i]["train_acc"] = []
        res["model_%s"%i]["val_acc"] = []
        best_valid_loss = float('inf')
        num_steps = 0
        ### reset model for each round
        # model.load_state_dict(init_param)
        model = copy.deepcopy(init_model)
        for name, param in model.named_parameters():                
            if name.startswith('bert'):
                param.requires_grad = False
        model = model.to(device)
        optimizer = optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()
        criterion = criterion.to(device)
        
        for epoch in range(N_EPOCHS):
            print('Begin epoch %s'%epoch)
            start_time = time.time()
            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
            end_time = time.time()
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), 'bert-model.pt')
                num_steps = 0
            else:
                num_steps += 1
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
            res["model_%s"%i]["time"].append(end_time - start_time)
            res["model_%s"%i]["train_loss"].append(train_loss)
            res["model_%s"%i]["val_loss"].append(valid_loss)
            res["model_%s"%i]["train_acc"].append(train_acc)
            res["model_%s"%i]["val_acc"].append(valid_acc)
            if num_steps >= early_stopping:
                break
        
        model.load_state_dict(torch.load('bert-model.pt'))
        test_loss, test_acc = evaluate(model, test_iterator, criterion)
        print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        res["test_loss"].append(test_loss)
        res["test_acc"].append(test_acc)
    return res

In [5]:
data_name = "TREC"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset_bert(data_name, device)

HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("bert_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.815 | Train Acc: 68.87%
	 Val. Loss: 0.363 |  Val. Acc: 88.72%
Begin epoch 1
	Train Loss: 0.390 | Train Acc: 86.25%
	 Val. Loss: 0.324 |  Val. Acc: 87.86%
Begin epoch 2
	Train Loss: 0.297 | Train Acc: 89.73%
	 Val. Loss: 0.332 |  Val. Acc: 89.77%
Begin epoch 3
	Train Loss: 0.251 | Train Acc: 91.39%
	 Val. Loss: 0.258 |  Val. Acc: 91.33%
Begin epoch 4
	Train Loss: 0.191 | Train Acc: 93.69%
	 Val. Loss: 0.292 |  Val. Acc: 91.17%
Begin epoch 5
	Train Loss: 0.157 | Train Acc: 94.59%
	 Val. Loss: 0.279 |  Val. Acc: 90.81%
Begin epoch 6
	Train Loss: 0.165 | Train Acc: 94.19%
	 Val. Loss: 0.322 |  Val. Acc: 89.42%
Begin epoch 7
	Train Loss: 0.113 | Train Acc: 96.09%
	 Val. Loss: 0.222 |  Val. Acc: 93.07%
Begin epoch 8
	Train Loss: 0.093 | Train Acc: 97.04%
	 Val. Loss: 0.249 |  Val. Acc: 92.53%
Begin epoch 9
	Train Loss: 0.091 | Train Acc: 96.80%
	 Val. Loss: 0.258 |  Val. Acc: 92.36%
Begin epoch 10
	Train Loss: 0.073 | Train Acc: 97.63%
	 V

In [6]:
data_name = "SST"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset_bert(data_name, device)

HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("bert_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.354 | Train Acc: 84.43%
	 Val. Loss: 0.287 |  Val. Acc: 88.04%
Begin epoch 1
	Train Loss: 0.278 | Train Acc: 88.40%
	 Val. Loss: 0.282 |  Val. Acc: 89.87%
Begin epoch 2
	Train Loss: 0.240 | Train Acc: 90.21%
	 Val. Loss: 0.251 |  Val. Acc: 91.25%
Begin epoch 3
	Train Loss: 0.214 | Train Acc: 91.53%
	 Val. Loss: 0.413 |  Val. Acc: 86.72%
Begin epoch 4
	Train Loss: 0.196 | Train Acc: 92.30%
	 Val. Loss: 0.268 |  Val. Acc: 90.69%
Begin epoch 5
	Train Loss: 0.182 | Train Acc: 92.96%
	 Val. Loss: 0.302 |  Val. Acc: 89.87%
Begin epoch 6
	Train Loss: 0.171 | Train Acc: 93.38%
	 Val. Loss: 0.262 |  Val. Acc: 89.98%
Begin epoch 7
	Train Loss: 0.163 | Train Acc: 93.67%
	 Val. Loss: 0.293 |  Val. Acc: 90.92%
Test Loss: 0.239 | Test Acc: 90.33%
Begin training model 1
Begin epoch 0
	Train Loss: 0.356 | Train Acc: 84.24%
	 Val. Loss: 0.249 |  Val. Acc: 90.40%
Begin epoch 1
	Train Loss: 0.276 | Train Acc: 88.46%
	 Val. Loss: 0.249 |  Val. Acc: 90.80

In [ ]:
data_name = "IMDB"
train_iterator, valid_iterator, test_iterator, TEXT, LABEL = load_dataset_bert(data_name, device)

HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

res = multi_models(model, train_iterator, valid_iterator, test_iterator)
res["num_param"] = count_parameters(model)
with open("bert_%s.json"%data_name, "w") as outfile:
    outfile.write(json.dumps(res, indent=4))

Begin training model 0
Begin epoch 0
	Train Loss: 0.439 | Train Acc: 78.33%
	 Val. Loss: 0.234 |  Val. Acc: 90.70%
Begin epoch 1
	Train Loss: 0.251 | Train Acc: 90.23%
	 Val. Loss: 0.214 |  Val. Acc: 91.29%
Begin epoch 2
	Train Loss: 0.222 | Train Acc: 91.34%
	 Val. Loss: 0.204 |  Val. Acc: 92.19%
Begin epoch 3
	Train Loss: 0.199 | Train Acc: 92.44%
	 Val. Loss: 0.216 |  Val. Acc: 92.11%
Begin epoch 4
	Train Loss: 0.174 | Train Acc: 93.43%
	 Val. Loss: 0.211 |  Val. Acc: 92.70%
Begin epoch 5
	Train Loss: 0.148 | Train Acc: 94.64%
	 Val. Loss: 0.216 |  Val. Acc: 92.77%
Begin epoch 6
	Train Loss: 0.128 | Train Acc: 95.26%
	 Val. Loss: 0.221 |  Val. Acc: 92.58%
Begin epoch 7
	Train Loss: 0.106 | Train Acc: 96.21%
	 Val. Loss: 0.252 |  Val. Acc: 92.42%
Test Loss: 0.199 | Test Acc: 92.15%
Begin training model 1
Begin epoch 0
	Train Loss: 0.402 | Train Acc: 80.88%
	 Val. Loss: 0.224 |  Val. Acc: 91.25%
Begin epoch 1
	Train Loss: 0.250 | Train Acc: 90.26%
	 Val. Loss: 0.261 |  Val. Acc: 89.61